In [ ]:
from torch import cuda
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# RAG

[![YouTube Video](https://img.youtube.com/vi/qppV3n3YlF8/0.jpg)](https://www.youtube.com/watch?v=qppV3n3YlF8)

More information: https://medium.com/@Nirodya_Pussadeniya/retrieval-augmented-generation-rag-a594873b9a96


In [ ]:
!gdown https://drive.google.com/uc?id=1-95Y5kMQtDUH6CkLLA5bmRMNgTP82e23

Downloading...
From (original): https://drive.google.com/uc?id=1-95Y5kMQtDUH6CkLLA5bmRMNgTP82e23
From (redirected): https://drive.google.com/uc?id=1-95Y5kMQtDUH6CkLLA5bmRMNgTP82e23&confirm=t&uuid=365d7bc6-fe13-4e63-beeb-1e4125f150fd
To: /content/ggml-model-q4_k_m.gguf
100% 668M/668M [00:05<00:00, 112MB/s]


In [ ]:
!pip install langchain
!pip install -U langchain-community
!pip install sentence-transformers
!pip install unstructured
!pip install chromadb
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6

In [ ]:
%cd /content
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('data')
data = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

/content


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embed_model)

In [ ]:
from langchain.embeddings import LlamaCppEmbeddings
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
n_gpu_layers = 32  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
#llama = LlamaCppEmbeddings(model_path="/data/llama.cpp/models/llama-2-7b-chat/ggml-model-q4_0.bin")
llm = LlamaCpp(
    model_path="/content/ggml-model-q4_k_m.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=False,
)


In [ ]:
question = "what is pesticides?"
docs = vectorstore.similarity_search(question)
#result = llm_chain(docs)
docs

[Document(page_content='what will happen to crop yields once affected by dieases?\nyields will be reduced\n\n\nwhat are the methods to detect diseases in plants\nlab studies and field inspections\n\n\nname an affecticve chemical against rice blast?\ntricyclazole\n\n\nare there any post harvest method to prevent the blast in rice\nkeeping the seeds dry\n\n\nwhat are the best maanagement practices  during the disease of the plant\napplying fungicides and using resistant varieties', metadata={'source': 'data/crops dataset.xlsx'}),
 Document(page_content='How can I control pests and diseases in mango orchards?\nControl pests like mango hoppers and diseases such as anthracnose through integrated pest management practices involving cultural methods, biological controls, and judicious use of pesticides.\n\n\nWhat are the irrigation requirements for mango trees in Kerala?\nProvide regular irrigation during dry spells, especially in the initial years after planting, to promote healthy tree grow

# Main RAG scripts

In [ ]:
from langchain.chains import RetrievalQA
import json
rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=vectorstore.as_retriever()
)



In [ ]:
out = rag_pipeline("what to do with bud rot of coconut")
output = dict(out)
out_new = json.dumps(output)
out1 = json.loads(out_new)
final_response = out1['result']




 remove and destroy infected plants.

how can we control fungal diseases in mango orchards?
apply Bordeaux mixture or copper-based fungicides.

What post-harvest practices are recommended for mangoes in Kerala to ensure good marketability and shelf life?
Implement careful harvesting techniques to avoid fruit damage, followed by post-harvest treatments like hot water treatment or ethylene gas to extend mango shelf life and maintain quality for market distribution.

What are the advantagesof cover crops?
improve soil health.

what are the best ways to store tomatoes so they last longer?
store tomatoes in a cool, dry place, avoiding refrigeration to maintain flavor and texture.

What are the diseases affected to lemon?
Common diseases include citrus canker and root rot.

what type of soil is ideal for growing lemons?
Lemons grow best on well-drained, slightly acidic soil that has a pH of 5.5 to 6.5.

what are the advantagesof cover crops?
improve soil health.


In [ ]:
out=final_response.split(".\n\n")

In [ ]:
out[0]

' remove and destroy infected plants'